In [3]:
import h5py
import socket
import capnp
import sys
import struct
from os.path import exists
import numpy as np

/home/dorian/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
!cat /home/franziska/Documents/Master/sem1/argos-zero/src/capnp/CapnpGame.capnp

cat: /home/franziska/Documents/Master/sem1/argos-zero/src/capnp/CapnpGame.capnp: No such file or directory


In [5]:
def recvall(sock, n):
    # Helper function to recv n bytes or return None if EOF is hit
    data = b''
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return data
        data += packet
    return data

In [6]:
class GameLogger:
    def __init__(self, port, filename, limited_game_num=1000):
        self.count_key = "count_id"
        self.dataset_key = "game_record"
        self.limited_game_num = limited_game_num
        
        self._init_socket(port)
        
        self.file_i = 1
        self.filename = filename
        self._init_h5()

    def _init_socket(self, port):
        self.server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server.bind(("localhost", PORT))
        self.server.listen(50)
        self.server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        
    def _init_h5(self):
        file = "%s-%.4d.h5" % (self.filename, self.file_i)
        self.f = h5py.File(file, "a")
                
        if self.dataset_key not in self.f.keys():
             # find the datatype of the np.void(msg) for creating dataset in the next step
            dt = np.dtype("V195880")
            # create dataset according to the datasetkey and set limit for num of games/ num of records
            gamerecord_dataset = self.f.create_dataset(self.dataset_key, (self.limited_game_num,) ,dtype=dt)
            # initialize the attribute count_id
            gamerecord_dataset.attrs[self.count_key] = 0
        
    def _write_h5(self, msg):                
        gamerecord_dataset = self.f[self.dataset_key]
        
        # the capnp msg will be directly stored in hdf5
        current_id = gamerecord_dataset.attrs[self.count_key]
        gamerecord_dataset[current_id] = msg
        gamerecord_dataset.attrs[self.count_key] += 1
        
        if self.limited_game_num <= gamerecord_dataset.attrs[self.count_key]:
            self.file_i += 1
            self.f.close()
            self._init_h5()
        
    def listen(self):
        try:
            sys.stdout.flush()
            
            while True:
                client, address = self.server.accept()
                msg = recvall(client, 4096 * 100)
                self._write_h5(msg)
                
        finally:
            self.server.close()
            self.f.close()
            print("closed")

In [7]:
PORT = 6000
logger = GameLogger(port=PORT, filename='game_record')
logger.listen()

closed


KeyboardInterrupt: 